# Aim of this notebook :
The aim of this notebook is to estimate :
* Vocals
* Bass
* Drums
* Other (rest of the accompaniment)
of the song you want using pretrained models from SigSep Open-Unmix.

### Steps :
1. Load the song with librosa
2. Apply model
3. Save and listen

# Imports

In [ ]:
import torch
import pickle
import stempeg
import librosa
import soundfile
import tensorflow
import numpy as np
from pathlib import Path
from openunmix import predict
from matplotlib import gridspec
import matplotlib.pyplot as plt
from scipy.io.wavfile import write
from scipy.signal import stft, istft
from IPython.display import Audio, display

## Variables

In [ ]:
SONGS_DIR = Path("songs")
SONG_DIR = SONGS_DIR/"WishYouWereHere"
path = SONG_DIR/"9-05 Wish You Were Here.mp3"

In [ ]:
# Utils functions

def save_estimates(data):
    filename = SONG_DIR/'estimates.pkl'
    outfile = open(str(filename),'wb')
    pickle.dump(estimates, outfile)
    outfile.close()

def load_estimates(path):
    infile = open(path,'rb')
    data = pickle.load(infile)
    infile.close()
    return data

def save_audio_files(estimates, rate):
    for target, estimate in estimates.items():
        track = estimate.detach().cpu().numpy()[0].transpose(1, 0)
        write(f"{SONG_DIR/target}.wav", rate, track)

## Load

In [ ]:
song, rate = librosa.load(path)

In [ ]:
# x = song[(1*60) * rate : (2*60) * rate]  # Crop the song (inference time consideration)
x = song
x = np.array([x, x])  # Reshape to stereo

## Apply model

In [ ]:
estimates = predict.separate(
    torch.tensor(x).float(),
    rate=rate,
    device="cpu"
)

## Save results

In [ ]:
save_estimates(estimates)

In [ ]:
save_audio_files(estimates, rate*2)

# Open Unmix model (INRIA) [PyTorch docs](https://pytorch.org/hub/sigsep_open-unmix-pytorch_umx/) :

![Diagram](sigsep_umx-diagram.png)

The SigSep repo provides a separation pipeline : the **Separator meta-model**
1. Compute the mixture's spectrogram using STFT : [Torchaudio transforms](https://pytorch.org/audio/stable/transforms.html)
2. Apply multiple spectrogram models (one for each desired target) [model implementation on Github](https://github.com/sigsep/open-unmix-pytorch/blob/master/openunmix/model.py)
3. Combine their outputs through a multichannel generalized Wiener filter (using [Norbert](https://github.com/sigsep/norbert))
4. Apply the inverse STFT using torchaudio.